(kalman)=
```{raw} jupyter
<div id="qe-notebook-header" align="right" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" width="250px" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>
```

# 卡尔曼滤波器的另一个视角

```{index} single: Kalman Filter 2
```

```{contents} Contents
:depth: 2
```

在之前的量化经济学讲座 {doc}`卡尔曼滤波器的初步介绍 <kalman>` 中，我们使用卡尔曼滤波器来估计火箭的位置。

在本讲座中，我们将使用卡尔曼滤波器来推断劳动者的：
1. 人力资本
2. 劳动者投入人力资本积累的努力程度

这两个变量都是公司无法直接观察到的。

公司只能通过观察劳动者产生的产出历史，以及理解这些产出如何依赖于劳动者的人力资本，以及人力资本如何作为劳动者努力程度的函数来演化，来了解这些信息。

我们将设定一个规则，说明公司如何根据每期获得的信息来支付劳动者工资。

除了Anaconda中的内容外，本讲座还需要以下库：

In [1]:
!pip install quantecon

为了进行模拟，我们引入以下导入，与 {doc}`卡尔曼滤波器的初步介绍 <kalman>` 相同：

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
FONTPATH = "fonts/SourceHanSerifSC-SemiBold.otf"
mpl.font_manager.fontManager.addfont(FONTPATH)
plt.rcParams['font.family'] = ['Source Han Serif SC']

import numpy as np
from quantecon import Kalman, LinearStateSpace
from collections import namedtuple
from scipy.stats import multivariate_normal
import matplotlib as mpl
# Configure Matplotlib to use pdfLaTeX and CJKutf8
mpl.rcParams.update({
    'text.usetex': True,
    'text.latex.preamble': r'''
        \usepackage{{CJKutf8}}
        \usepackage{{amsmath}}
    '''
})

# Function to wrap Chinese text in CJK environment
def cjk(text):
    return rf'\begin{{CJK}}{{UTF8}}{{gbsn}}{text}\end{{CJK}}'

## 劳动者的产出

一个代表性劳动者永久受雇于一家公司。

劳动者的产出由以下动态过程描述：

```{math}
:label: worker_model

\begin{aligned}
h_{t+1} &= \alpha h_t + \beta u_t + c w_{t+1}, \quad c_{t+1} \sim {\mathcal N}(0,1) \\
u_{t+1} & = u_t \\
y_t & = g h_t + v_t , \quad v_t \sim {\mathcal N} (0, R)
\end{aligned}
```

其中：

* $h_t$ 是时间 $t$ 时的人力资本对数
* $u_t$ 是时间 $t$ 时劳动者投入人力资本积累的努力程度的对数
* $y_t$ 是时间 $t$ 时劳动者产出的对数
* $h_0 \sim {\mathcal N}(\hat h_0, \sigma_{h,0})$
* $u_0 \sim {\mathcal N}(\hat u_0, \sigma_{u,0})$

模型的参数包括 $\alpha, \beta, c, R, g, \hat h_0, \hat u_0, \sigma_h, \sigma_u$。

在时间 $0$，公司雇佣了劳动者。

劳动者永久依附于公司，因此在所有时间 $t =0, 1, 2, \ldots$ 都为同一家公司工作。

在时间 $0$ 开始时，公司既无法观察到劳动者的初始人力资本 $h_0$，也无法观察到其固有的永久努力水平 $u_0$。

公司认为特定劳动者的 $u_0$ 服从高斯概率分布，因此由 $u_0 \sim {\mathcal N}(\hat u_0, \sigma_{u,0})$ 描述。

劳动者"类型"中的 $h_t$ 部分随时间变化，但努力程度部分 $u_t = u_0$ 保持不变。

这意味着从公司的角度来看，劳动者的努力程度实际上是一个未知的固定"参数"。

在时间 $t\geq 1$，对于特定劳动者，公司观察到 $y^{t-1} = [y_{t-1}, y_{t-2}, \ldots, y_0]$。

公司无法观察到劳动者的"类型" $(h_0, u_0)$。

但公司确实观察到劳动者在时间 $t$ 的产出 $y_t$，并记得劳动者的过去产出 $y^{t-1}$。

## 公司的工资设定政策

基于公司在时间 $t \geq 1$ 获得的关于劳动者的信息，公司支付劳动者的对数工资为：

$$
w_t = g  E [ h_t | y^{t-1} ], \quad t \geq 1
$$

在时间 $0$，支付劳动者的对数工资等于 $y_0$ 的无条件均值：

$$
w_0 = g \hat h_0
$$

在使用这个支付规则时，公司考虑到劳动者今天的对数产出部分来自完全由运气决定的随机成分 $v_t$，并且假设 $v_t$ 与 $h_t$ 和 $u_t$ 独立。

## 状态空间表示

将系统 [](worker_model) 写成状态空间形式：

```{math}
\begin{aligned}
\begin{bmatrix} h_{t+1} \cr u_{t+1} \end{bmatrix} &= \begin{bmatrix} \alpha & \beta \cr 0 & 1 \end{bmatrix}\begin{bmatrix} h_{t} \cr u_{t} \end{bmatrix} + \begin{bmatrix} c \cr 0 \end{bmatrix} w_{t+1} \cr
y_t & = \begin{bmatrix} g & 0 \end{bmatrix} \begin{bmatrix} h_{t} \cr u_{t} \end{bmatrix} + v_t
\end{aligned}
```

这等价于：

```{math}
:label: ssrepresent
\begin{aligned} 
x_{t+1} & = A x_t + C w_{t+1} \cr
y_t & = G x_t + v_t \cr
x_0 & \sim {\mathcal N}(\hat x_0, \Sigma_0) 
\end{aligned}
```

其中：

```{math}
x_t  = \begin{bmatrix} h_{t} \cr u_{t} \end{bmatrix} , \quad
\hat x_0  = \begin{bmatrix} \hat h_0 \cr \hat u_0 \end{bmatrix} , \quad
\Sigma_0  = \begin{bmatrix} \sigma_{h,0} & 0 \cr
                     0 & \sigma_{u,0} \end{bmatrix}
```

为了计算公司的工资设定政策，我们首先创建一个 `namedtuple` 来存储模型的参数：

In [3]:
WorkerModel = namedtuple("WorkerModel", 
                ('A', 'C', 'G', 'R', 'xhat_0', 'Σ_0'))

def create_worker(α=.8, β=.2, c=.2,
                  R=.5, g=1.0, hhat_0=4, uhat_0=4, 
                  σ_h=4, σ_u=4):
    
    A = np.array([[α, β], 
                  [0, 1]])
    C = np.array([[c], 
                  [0]])
    G = np.array([g, 1])

    # 定义初始状态和协方差矩阵
    xhat_0 = np.array([[hhat_0], 
                       [uhat_0]])
    
    Σ_0 = np.array([[σ_h, 0],
                    [0, σ_u]])
    
    return WorkerModel(A=A, C=C, G=G, R=R, xhat_0=xhat_0, Σ_0=Σ_0)

请注意 `WorkerModel` namedtuple 如何创建计算相关状态空间表示 {eq}`ssrepresent` 所需的所有对象。

这很方便，因为为了模拟劳动者的历史 $\{y_t, h_t\}$，我们需要使用 [`LinearStateSpace`](https://quanteconpy.readthedocs.io/en/latest/tools/lss.html) 类为他/她形成状态空间系统。

In [4]:
# 定义 A, C, G, R, xhat_0, Σ_0
worker = create_worker()
A, C, G, R = worker.A, worker.C, worker.G, worker.R
xhat_0, Σ_0 = worker.xhat_0, worker.Σ_0

# 创建 LinearStateSpace 对象
ss = LinearStateSpace(A, C, G, np.sqrt(R), 
        mu_0=xhat_0, Sigma_0=np.zeros((2,2)))

T = 100
x, y = ss.simulate(T)
y = y.flatten()

h_0, u_0 = x[0, 0], x[1, 0]

接下来，为了计算公司基于其获得的关于劳动者的信息来设定对数工资的政策，我们使用本量化经济学讲座 {doc}`卡尔曼滤波器的初步介绍 <kalman>` 中描述的卡尔曼滤波器。

特别是，我们想要计算"创新表示"中的所有对象。

## 创新表示

我们已经掌握了形成劳动者产出过程 $\{y_t\}_{t=0}^T$ 的创新表示所需的所有对象。

让我们现在编写代码：

```{math}
\begin{aligned}
\hat x_{t+1} & = A \hat x_t + K_t a_t \cr
y_{t} & = G \hat x_t + a_t
\end{aligned}
```
其中 $K_t$ 是时间 $t$ 的卡尔曼增益矩阵。

我们使用 [`Kalman`](https://quanteconpy.readthedocs.io/en/latest/tools/kalman.html) 类来完成这个任务：

In [5]:
kalman = Kalman(ss, xhat_0, Σ_0)
Σ_t = np.zeros((*Σ_0.shape, T-1))
y_hat_t = np.zeros(T-1)
x_hat_t = np.zeros((2, T-1))

for t in range(1, T):
    kalman.update(y[t])
    x_hat, Σ = kalman.x_hat, kalman.Sigma
    Σ_t[:, :, t-1] = Σ
    x_hat_t[:, t-1] = x_hat.reshape(-1)
    y_hat_t[t-1] = worker.G @ x_hat

x_hat_t = np.concatenate((x[:, 1][:, np.newaxis], 
                    x_hat_t), axis=1)
Σ_t = np.concatenate((worker.Σ_0[:, :, np.newaxis], 
                    Σ_t), axis=2)
u_hat_t = x_hat_t[1, :]

/tmp/ipykernel_6483/2927621375.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_hat_t[t-1] = worker.G @ x_hat


对于 $h_0, u_0$ 的一个实现，我们绘制 $E y_t = G \hat x_t $，其中 $\hat x_t = E [x_t | y^{t-1}]$。

我们还绘制 $E [u_0 | y^{t-1}]$，这是公司基于其拥有的信息 $y^{t-1}$ 对劳动者固有的"工作伦理" $u_0$ 的推断。

我们可以观察公司对劳动者工作伦理的推断 $E [u_0 | y^{t-1}]$ 如何逐渐收敛于隐藏的 $u_0$，而 $u_0$ 是公司无法直接观察到的。

In [6]:
fig, ax = plt.subplots(1, 2)

ax[0].plot(y_hat_t, label=r'$E[y_t| y^{t-1}]$')
ax[0].set_xlabel(cjk('时间'))
ax[0].set_ylabel(r'$E[y_t]$')
ax[0].set_title(cjk('$E[y_t]$ 随时间变化'))
ax[0].legend()

ax[1].plot(u_hat_t, label=r'$E[u_t|y^{t-1}]$')
ax[1].axhline(y=u_0, color='grey', 
            linestyle='dashed', label=fr'$u_0={u_0:.2f}$')
ax[1].set_xlabel(cjk('时间'))
ax[1].set_ylabel(r'$E[u_t|y^{t-1}]$')
ax[1].set_title(cjk('推断的工作伦理随时间变化'))
ax[1].legend()

fig.tight_layout()
plt.show()

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp7no0wvhd c47aad118656eb7c905401f2906e87a5.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./c47aad118656eb7c905401f2906e87a5.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-01-22>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/t

Error in callback <function _draw_all_if_interactive at 0x7f9fa335a660> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp87px4f45 c47aad118656eb7c905401f2906e87a5.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./c47aad118656eb7c905401f2906e87a5.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-01-22>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/t

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpwc49c69g c47aad118656eb7c905401f2906e87a5.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./c47aad118656eb7c905401f2906e87a5.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-01-22>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/t

<Figure size 640x480 with 2 Axes>

## 一些计算实验

让我们看看 $\Sigma_0$ 和 $\Sigma_T$，以了解公司在设定的时间范围内对隐藏状态了解多少。

In [ ]:
print(Σ_t[:, :, 0])

In [ ]:
print(Σ_t[:, :, -1])

显然，条件协方差矩阵中的元素随时间变小。

通过在不同时间 $t$ 绘制 $E [x_t |y^{t-1}] $ 周围的置信椭圆，我们可以生动地展示条件协方差矩阵 $\Sigma_t$ 如何演化。

In [ ]:
# 创建用于等高线绘制的点网格
h_range = np.linspace(x_hat_t[0, :].min()-0.5*Σ_t[0, 0, 1], 
                      x_hat_t[0, :].max()+0.5*Σ_t[0, 0, 1], 100)
u_range = np.linspace(x_hat_t[1, :].min()-0.5*Σ_t[1, 1, 1], 
                      x_hat_t[1, :].max()+0.5*Σ_t[1, 1, 1], 100)
h, u = np.meshgrid(h_range, u_range)

# 为每个时间步创建子图
fig, axs = plt.subplots(1, 3, figsize=(12, 7))

# 遍历每个时间步
for i, t in enumerate(np.linspace(0, T-1, 3, dtype=int)):
    # 创建具有时间步 t 的 x_hat 和 Σ 的多变量正态分布
    mu = x_hat_t[:, t]
    cov = Σ_t[:, :, t]
    mvn = multivariate_normal(mean=mu, cov=cov)
    
    # 在网格上评估多变量正态 PDF
    pdf_values = mvn.pdf(np.dstack((h, u)))
    
    # 创建 PDF 的等高线图
    con = axs[i].contour(h, u, pdf_values, cmap='viridis')
    axs[i].clabel(con, inline=1, fontsize=10)
    axs[i].set_title(cjk('时间步')+f'{t+1}')
    axs[i].set_xlabel(r'$h_{{{}}}$'.format(str(t+1)))
    axs[i].set_ylabel(r'$u_{{{}}}$'.format(str(t+1)))
    
    cov_latex = r'$\Sigma_{{{}}}= \begin{{bmatrix}} {:.2f} & {:.2f} \\ {:.2f} & {:.2f} \end{{bmatrix}}$'.format(
        t+1, cov[0, 0], cov[0, 1], cov[1, 0], cov[1, 1]
    )
    axs[i].text(0.33, -0.15, cov_latex, transform=axs[i].transAxes)

    
plt.tight_layout()
plt.show()

注意证据 $y^t$ 的积累如何随着样本量 $t$ 的增长影响置信椭圆的形状。

现在让我们使用我们的代码将隐藏状态 $x_0$ 设置为特定的向量，以观察公司如何从我们感兴趣的某个 $x_0$ 开始学习。

例如，让我们说 $h_0 = 0$ 和 $u_0 = 4$。

这是实现这个目标的一种方式：

In [ ]:
# 例如，我们可能想要 h_0 = 0 和 u_0 = 4
mu_0 = np.array([0.0, 4.0])

# 创建一个 LinearStateSpace 对象，其中 Sigma_0 为零矩阵
ss_example = LinearStateSpace(A, C, G, np.sqrt(R), mu_0=mu_0, 
                              # 这行强制 h_0=0 和 u_0=4
                              Sigma_0=np.zeros((2, 2))
                             )

T = 100
x, y = ss_example.simulate(T)
y = y.flatten()

# 现在 h_0=0 和 u_0=4
h_0, u_0 = x[0, 0], x[1, 0]
print('h_0 =', h_0)
print('u_0 =', u_0)

实现相同目标的另一种方式是使用以下代码：

In [ ]:
# 如果我们想要设置初始 
# h_0 = hhat_0 = 0 和 u_0 = uhhat_0 = 4.0:
worker = create_worker(hhat_0=0.0, uhat_0=4.0)

ss_example = LinearStateSpace(A, C, G, np.sqrt(R), 
                              # 这行取 h_0=hhat_0 和 u_0=uhhat_0
                              mu_0=worker.xhat_0,
                              # 这行强制 h_0=hhat_0 和 u_0=uhhat_0
                              Sigma_0=np.zeros((2, 2))
                             )

T = 100
x, y = ss_example.simulate(T)
y = y.flatten()

# 现在 h_0 和 u_0 将精确等于 hhat_0
h_0, u_0 = x[0, 0], x[1, 0]
print('h_0 =', h_0)
print('u_0 =', u_0)

对于这个劳动者，让我们生成一个类似上面的图：

In [ ]:
# 首先我们使用初始 xhat_0 和 Σ_0 计算卡尔曼滤波器
kalman = Kalman(ss, xhat_0, Σ_0)
Σ_t = []
y_hat_t = np.zeros(T-1)
u_hat_t = np.zeros(T-1)

# 然后我们使用基于上述线性状态模型的观测值 y 
# 迭代更新卡尔曼滤波器类：
for t in range(1, T):
    kalman.update(y[t])
    x_hat, Σ = kalman.x_hat, kalman.Sigma
    Σ_t.append(Σ)
    y_hat_t[t-1] = worker.G @ x_hat
    u_hat_t[t-1] = x_hat[1]


# 生成 y_hat_t 和 u_hat_t 的图
fig, ax = plt.subplots(1, 2)

ax[0].plot(y_hat_t, label=r'$E[y_t| y^{t-1}]$')
ax[0].set_xlabel(cjk('时间'))
ax[0].set_ylabel(r'$E[y_t]$')
ax[0].set_title(cjk('$E[y_t]$ 随时间变化'))
ax[0].legend()

ax[1].plot(u_hat_t, label=r'$E[u_t|y^{t-1}]$')
ax[1].axhline(y=u_0, color='grey', 
            linestyle='dashed', label=fr'$u_0={u_0:.2f}$')
ax[1].set_xlabel(cjk('时间'))
ax[1].set_ylabel(r'$E[u_t|y^{t-1}]$')
ax[1].set_title(cjk('推断的工作伦理随时间变化'))
ax[1].legend()

fig.tight_layout()
plt.show()

更一般地，我们可以在 `create_worker` namedtuple 中更改定义劳动者的部分或全部参数。

这是一个例子：

In [ ]:
# 我们可以在创建劳动者时设置这些参数 -- 就像类一样！
hard_working_worker =  create_worker(α=.4, β=.8, 
                        hhat_0=7.0, uhat_0=100, σ_h=2.5, σ_u=3.2)

print(hard_working_worker)

我们还可以为不同的劳动者模拟 $T = 50$ 期的系统。

推断的工作伦理和真实工作伦理之间的差异随时间收敛到 $0$。

这表明滤波器正在逐渐教会劳动者和公司关于劳动者努力程度的信息。

In [ ]:
def simulate_workers(worker, T, ax, mu_0=None, Sigma_0=None, 
                    diff=True, name=None, title=None):
    A, C, G, R = worker.A, worker.C, worker.G, worker.R
    xhat_0, Σ_0 = worker.xhat_0, worker.Σ_0
    
    if isinstance(mu_0, type(None)):
        mu_0 = xhat_0
    if isinstance(Sigma_0, type(None)):
        Sigma_0 = worker.Σ_0
        
    ss = LinearStateSpace(A, C, G, np.sqrt(R), 
                        mu_0=mu_0, Sigma_0=Sigma_0)

    x, y = ss.simulate(T)
    y = y.flatten()

    u_0 = x[1, 0]
    
    # 计算卡尔曼滤波器
    kalman = Kalman(ss, xhat_0, Σ_0)
    Σ_t = []
    
    y_hat_t = np.zeros(T)
    u_hat_t = np.zeros(T)

    for i in range(T):
        kalman.update(y[i])
        x_hat, Σ = kalman.x_hat, kalman.Sigma
        Σ_t.append(Σ)
        y_hat_t[i] = worker.G @ x_hat
        u_hat_t[i] = x_hat[1]

    if diff == True:
        title = (cjk('推断的工作伦理与真实工作伦理的差异随时间变化') 
                 if title == None else title)
        
        ax.plot(u_hat_t - u_0, alpha=.5)
        ax.axhline(y=0, color='grey', linestyle='dashed')
        ax.set_xlabel(cjk('时间'))
        ax.set_ylabel(r'$E[u_t|y^{t-1}] - u_0$')
        ax.set_title(title)
        
    else:
        label_line = (r'$E[u_t|y^{t-1}]$' if name == None 
                      else name)
        title = (cjk('推断的工作伦理随时间变化')
                if title == None else title)
        
        u_hat_plot = ax.plot(u_hat_t, label=label_line)
        ax.axhline(y=u_0, color=u_hat_plot[0].get_color(), 
                    linestyle='dashed', alpha=0.5)
        ax.set_xlabel(cjk('时间'))
        ax.set_ylabel(r'$E[u_t|y^{t-1}]$')
        ax.set_title(title)

In [ ]:
num_workers = 3
T = 50
fig, ax = plt.subplots(figsize=(7, 7))

for i in range(num_workers):
    worker = create_worker(uhat_0=4+2*i)
    simulate_workers(worker, T, ax)
ax.set_ylim(ymin=-2, ymax=2)
plt.show()

In [ ]:
# 我们还可以生成 u_t 的图：

T = 50
fig, ax = plt.subplots(figsize=(7, 7))

uhat_0s = [2, -2, 1]
αs = [0.2, 0.3, 0.5]
βs = [0.1, 0.9, 0.3]

for i, (uhat_0, α, β) in enumerate(zip(uhat_0s, αs, βs)):
    worker = create_worker(uhat_0=uhat_0, α=α, β=β)
    simulate_workers(worker, T, ax,
                    # 通过设置 diff=False，它将给出 u_t
                    diff=False, name=r'$u_{{{}, t}}$'.format(i))
    
ax.axhline(y=u_0, xmin=0, xmax=0, color='grey', 
           linestyle='dashed', label=r'$u_{i, 0}$')
ax.legend(bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# 我们还可以为所有劳动者使用精确的 u_0=1 和 h_0=2

T = 50
fig, ax = plt.subplots(figsize=(7, 7))

# 这两行设置所有劳动者的 u_0=1 和 h_0=2
mu_0 = np.array([[1],
                 [2]])
Sigma_0 = np.zeros((2,2))

uhat_0s = [2, -2, 1]
αs = [0.2, 0.3, 0.5]
βs = [0.1, 0.9, 0.3]

for i, (uhat_0, α, β) in enumerate(zip(uhat_0s, αs, βs)):
    worker = create_worker(uhat_0=uhat_0, α=α, β=β)
    simulate_workers(worker, T, ax, mu_0=mu_0, Sigma_0=Sigma_0, 
                     diff=False, name=r'$u_{{{}, t}}$'.format(i))
    
# 这控制图的边界
ax.set_ylim(ymin=-3, ymax=3)
ax.axhline(y=u_0, xmin=0, xmax=0, color='grey', 
           linestyle='dashed', label=r'$u_{i, 0}$')
ax.legend(bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# 我们可以只为其中一个劳动者生成图：

T = 50
fig, ax = plt.subplots(figsize=(7, 7))

mu_0_1 = np.array([[1],
                 [100]])
mu_0_2 = np.array([[1],
                 [30]])
Sigma_0 = np.zeros((2,2))

uhat_0s = 100
αs = 0.5
βs = 0.3

worker = create_worker(uhat_0=uhat_0, α=α, β=β)
simulate_workers(worker, T, ax, mu_0=mu_0_1, Sigma_0=Sigma_0, 
                 diff=False, name=cjk('勤奋的劳动者'))
simulate_workers(worker, T, ax, mu_0=mu_0_2, Sigma_0=Sigma_0, 
                 diff=False, 
                 title=cjk('一个勤奋的劳动者和一个不太勤奋的劳动者'),
                 name=cjk('普通劳动者'))
ax.axhline(y=u_0, xmin=0, xmax=0, color='grey', 
           linestyle='dashed', label=r'$u_{i, 0}$')
ax.legend(bbox_to_anchor=(1, 0.5))
plt.show()

## 未来扩展

我们可以通过创建新类型的劳动者，并让公司仅通过观察他们的产出历史来了解他们的隐藏（对公司来说）状态，来进行许多富有启发性的实验。